<a href="https://colab.research.google.com/github/GalalMohammed/SHAI_Training_2022-b_Tasks/blob/main/Task_6_exercises_answered.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

![](logo1.jpg)

# **shAI Training 2021 | Level 1**

## Task #6 (End-to-End ML Project {part_2})

## Welcome to the exercises for reviewing second part of end to end ML project.
**Make sure that you read and understand ch2 from the hands-on ML book (page 72 to the end of the chapter ) before start with this notebook.**

**If you stuck with anything reread that part from the book and feel free to ask about anything in the messenger group as you go along.**

 ## Good Luck : )

## first run the following cell for the first part of the project to continue your work 

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 
import seaborn as sns
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer

In [2]:
import os
import tarfile
import urllib
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
HOUSING_PATH = os.path.join("datasets", "housing")
HOUSING_URL = DOWNLOAD_ROOT + "datasets/housing/housing.tgz"

def fetch_housing_data(housing_url=HOUSING_URL, housing_path=HOUSING_PATH):
    os.makedirs(housing_path, exist_ok=True)
    tgz_path = os.path.join(housing_path, "housing.tgz")
    urllib.request.urlretrieve(housing_url, tgz_path)
    housing_tgz = tarfile.open(tgz_path)
    housing_tgz.extractall(path=housing_path)
    housing_tgz.close()
    
def load_housing_data(housing_path=HOUSING_PATH):
   csv_path = os.path.join(housing_path, "housing.csv")
   return pd.read_csv(csv_path)
   
fetch_housing_data()
housing = load_housing_data()

rooms_ix, bedrooms_ix, population_ix, household_ix = [
    list(housing.columns).index(col)
    for col in ("total_rooms", "total_bedrooms", "population", "households")]

class CombinedAttributesAdder(BaseEstimator, TransformerMixin):
    def __init__(self, add_bedrooms_per_room = True):
        self.add_bedrooms_per_room = add_bedrooms_per_room
    def fit(self, X, y=None):
        return self  # nothing else to do
    def transform(self, X, y=None):
        rooms_per_household = X[:, rooms_ix] / X[:, household_ix]
        population_per_household = X[:, population_ix] / X[:, household_ix]
        if self.add_bedrooms_per_room:
            bedrooms_per_room = X[:, bedrooms_ix] / X[:, rooms_ix]
            return np.c_[X, rooms_per_household, population_per_household,
                         bedrooms_per_room]
        else:
            return np.c_[X, rooms_per_household, population_per_household]
        
train_set, test_set = train_test_split(housing, test_size=0.2, random_state=42)
housing = train_set.drop("median_house_value", axis=1)
housing_labels = train_set["median_house_value"].copy()

housing_num = housing.drop("ocean_proximity", axis=1)
num_attribs = list(housing_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
 ('imputer', SimpleImputer(strategy="median")),
 ('attribs_adder', CombinedAttributesAdder()),
 ('std_scaler', StandardScaler())])

full_pipeline = ColumnTransformer([
 ("num", num_pipeline, num_attribs),
 ("cat", OneHotEncoder(), cat_attribs)])

housing_prepared = full_pipeline.fit_transform(housing)

# 1- Select and Train a Model

# Let’s first train a LinearRegression model 

In [10]:
# CODE HERE
from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(housing_prepared,housing_labels)

LinearRegression()

# First try it out on a few instances from the training set:


In [12]:
housing_preparedDf = pd.DataFrame(housing_prepared)
some_data = housing_preparedDf.iloc[:5]
some_labels = housing_labels.iloc[:5]

In [13]:
# CODE HERE
housing_predictions = lin_reg.predict(some_data)
housing_predictions

array([181746.54359616, 290558.74973505, 244957.50017771, 146498.51061398,
       163230.42393939])

# measure this regression model’s RMSE on the whole training set 
* using Scikit-Learn’s mean_squared_error() function:

In [14]:
from sklearn.metrics import mean_squared_error

In [15]:
# CODE HERE
lin_mse = mean_squared_error(housing_labels, lin_reg.predict(housing_prepared))
lin_rmse = np.sqrt(lin_mse)
lin_rmse

67593.20745775253

# judge on the RMSE result for this model 
write down your answar 

In [19]:
lin_rmse / train_set["median_house_value"].mean()

0.32623039827101735

your answer goes here:
The error rate is about 32.62%, which seems bad.

# Let’s train a Decision Tree Regressor model 
## more powerful model

In [20]:
from sklearn.tree import DecisionTreeRegressor 

In [21]:
# CODE HERE
tree_reg = DecisionTreeRegressor()
tree_reg.fit(housing_prepared,housing_labels)

DecisionTreeRegressor()

# Now evaluate the model on the training set 
* using Scikit-Learn’s mean_squared_error() function:

In [22]:
# CODE HERE
housing_predictions = tree_reg.predict(housing_prepared)
tree_mse = mean_squared_error(housing_labels,housing_predictions)
tree_rmse = np.sqrt(tree_mse)
tree_rmse

0.0

# Explaine this result 
write down your answar

your answer goes here: The error rate is 0.0%, which means that the model overfits the data.

# Evaluation Using Cross-Validation

1-split the training set into 10 distinct subsets then train and evaluate the Decision Tree model

In [24]:
from sklearn.model_selection import cross_val_score

In [26]:
# CODE HERE
scores = cross_val_score(tree_reg, housing_prepared, housing_labels, scoring ="neg_mean_squared_error",cv = 10)
tree_rmse_scores = np.sqrt(-scores)

2- display the resultant scores and calculate its Mean and Standard deviation

In [27]:
# CODE HERE
print("Scores: ", tree_rmse_scores)
print("Mean: ", tree_rmse_scores.mean())
print("Standard Deviation: ", tree_rmse_scores.std())

Scores:  [64521.54879006 71215.53128988 68357.02591842 71818.32000758
 72947.32787772 66420.12632225 67430.46151279 68762.05951681
 66288.19301024 70954.29196713]
Mean:  68871.48862128636
Standard Deviation:  2628.434177844391


3-repaet the same steps to compute the same scores for the Linear Regression  model 

*notice the difference between the results of the two models*

In [28]:
# CODE HERE
scores = cross_val_score(lin_reg, housing_prepared, housing_labels, scoring ="neg_mean_squared_error",cv = 10)
lin_rmse_scores = np.sqrt(-scores)
print("Scores: ", lin_rmse_scores)
print("Mean: ", lin_rmse_scores.mean())
print("Standard Deviation: ", lin_rmse_scores.std())

Scores:  [65000.67382615 70960.56056304 67122.63935124 66089.63153865
 68402.54686442 65266.34735288 65218.78174481 68525.46981754
 72739.87555996 68957.34111906]
Mean:  67828.38677377408
Standard Deviation:  2468.091395065227


## Let’s train one last model the RandomForestRegressor.

In [29]:
# CODE HERE
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(housing_prepared, housing_labels)

RandomForestRegressor()

# repeat the same steps to compute the same scores its Mean and Standard deviation for the Random Forest model

In [30]:
# CODE HERE
scores = cross_val_score(forest_reg, housing_prepared, housing_labels, scoring ="neg_mean_squared_error",cv = 10)
forest_rmse_scores = np.sqrt(-scores)
print("Scores: ", forest_rmse_scores)
print("Mean: ", forest_rmse_scores.mean())
print("Standard Deviation: ", forest_rmse_scores.std())

Scores:  [46963.95907155 51703.50340411 49524.53181078 52077.45328745
 52776.36513193 47110.69481577 47329.39179148 50666.91830614
 49060.72793057 50337.5436055 ]
Mean:  49755.108915526806
Standard Deviation:  2017.2014729563653


# Save every model you experiment with 
*using the joblib library*

In [31]:
# CODE HERE
import joblib
lin_filename = 'finalized_lin_model.sav'
tree_filename = 'finalized_tree_model.sav'
forest_filename = 'finalized_forest_model.sav'
joblib.dump(lin_reg, lin_filename)
joblib.dump(tree_reg, tree_filename)
joblib.dump(forest_reg, forest_filename)

['finalized_forest_model.sav']

## now you have a shortlist of promising models. You now need to
## fine-tune them!
# Fine-Tune Your Model

## 1- Grid Search
## evaluate all the possible combinations of hyperparameter values for the RandomForestRegressor 
*It may take a long time*

In [32]:
from sklearn.model_selection import GridSearchCV

In [33]:
# CODE HERE
param_grid = [{'n_estimators': [3,10,30], 'max_features':[2,4,6,8]},{'bootstrap':[False], 'max_features':[2,3,4],'n_estimators':[3,10]}]
grid_search = GridSearchCV(forest_reg,param_grid,cv = 10, scoring = 'neg_mean_squared_error',return_train_score = True)
grid_search.fit(housing_prepared, housing_labels)

GridSearchCV(cv=10, estimator=RandomForestRegressor(),
             param_grid=[{'max_features': [2, 4, 6, 8],
                          'n_estimators': [3, 10, 30]},
                         {'bootstrap': [False], 'max_features': [2, 3, 4],
                          'n_estimators': [3, 10]}],
             return_train_score=True, scoring='neg_mean_squared_error')

with the evaluation scores

In [34]:
# CODE HERE
print("Best Params: ", grid_search.best_params_)
print("Best Estimator: ", grid_search.best_estimator_)
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres["mean_test_score"],cvres["params"]):
  print(np.sqrt(-mean_score), params)

Best Params:  {'max_features': 8, 'n_estimators': 30}
Best Estimator:  RandomForestRegressor(max_features=8, n_estimators=30)
62992.58468320721 {'max_features': 2, 'n_estimators': 3}
54899.39593514671 {'max_features': 2, 'n_estimators': 10}
52068.14947306645 {'max_features': 2, 'n_estimators': 30}
59325.35014693786 {'max_features': 4, 'n_estimators': 3}
52016.05203051994 {'max_features': 4, 'n_estimators': 10}
49678.153962681965 {'max_features': 4, 'n_estimators': 30}
58045.875296616316 {'max_features': 6, 'n_estimators': 3}
51822.687725097625 {'max_features': 6, 'n_estimators': 10}
49564.00894682268 {'max_features': 6, 'n_estimators': 30}
58007.11258833583 {'max_features': 8, 'n_estimators': 3}
51387.566910479305 {'max_features': 8, 'n_estimators': 10}
49511.0507620163 {'max_features': 8, 'n_estimators': 30}
60825.14514189835 {'bootstrap': False, 'max_features': 2, 'n_estimators': 3}
53628.85160282356 {'bootstrap': False, 'max_features': 2, 'n_estimators': 10}
58756.82262915244 {'boot

# Analyze the Best Models and Their Errors
1-indicate the relative importance of each attribute

In [35]:
# CODE HERE
grid_search.best_estimator_.feature_importances_

array([7.51182308e-02, 6.89335980e-02, 4.18258705e-02, 1.48267886e-02,
       1.35384706e-02, 1.40270435e-02, 1.34173287e-02, 3.54754132e-01,
       6.87449650e-02, 1.10342829e-01, 4.69619355e-02, 9.15753618e-03,
       1.61397622e-01, 1.07996691e-04, 2.28338328e-03, 4.56227068e-03])

2-display these importance scores next to their corresponding attribute names:

In [36]:
housing

,longitude,latitude,housing_median_age,total_rooms,total_bedrooms,population,households,median_income,ocean_proximity
14196,-117.03,32.71,33.0,3126.0,627.0,2300.0,623.0,3.2596,NEAR OCEAN
8267,-118.16,33.77,49.0,3382.0,787.0,1314.0,756.0,3.8125,NEAR OCEAN
17445,-120.48,34.66,4.0,1897.0,331.0,915.0,336.0,4.1563,NEAR OCEAN
14265,-117.11,32.69,36.0,1421.0,367.0,1418.0,355.0,1.9425,NEAR OCEAN
2271,-119.80,36.78,43.0,2382.0,431.0,874.0,380.0,3.5542,INLAND
...,...,...,...,...,...,...,...,...,...
11284,-117.96,33.78,35.0,1330.0,201.0,658.0,217.0,6.3700,<1H OCEAN
11964,-117.43,34.02,33.0,3084.0,570.0,1753.0,449.0,3.0500,INLAND
5390,-118.38,34.03,36.0,2101.0,569.0,1756.0,527.0,2.9344,<1H OCEAN
860,-121.96,37.58,15.0,3575.0,597.0,1777.0,559.0,5.7192,<1H OCEAN


In [37]:
# CODE HERE
extra_attribs = ["population_per_household","bedroom_per_room","rooms_per_household"]
num_attribs = list(housing_num.columns)
attributes = num_attribs + extra_attribs + cat_attribs
sorted(zip(grid_search.best_estimator_.feature_importances_,attributes),reverse = True)

[(0.35475413158869684, 'median_income'),
 (0.11034282850608955, 'bedroom_per_room'),
 (0.07511823083821832, 'longitude'),
 (0.06893359797611778, 'latitude'),
 (0.06874496498697129, 'population_per_household'),
 (0.046961935517122945, 'rooms_per_household'),
 (0.04182587047540786, 'housing_median_age'),
 (0.014826788562989586, 'total_rooms'),
 (0.014027043459046712, 'population'),
 (0.01353847056551243, 'total_bedrooms'),
 (0.013417328749069977, 'households'),
 (0.009157536179542913, 'ocean_proximity')]

## Now is the time to evaluate the final model on the test set.
# Evaluate Your System on the Test Set

1-get the predictors and the labels from your test set

In [43]:
# CODE HERE
Y_test = test_set.drop("median_house_value", axis=1)
Y_test_labels = test_set["median_house_value"].copy()

2-run your full_pipeline to transform the data

In [44]:
# CODE HERE
Y_test_num = Y_test.drop("ocean_proximity", axis=1)
num_attribs = list(Y_test_num)
cat_attribs = ["ocean_proximity"]

num_pipeline = Pipeline([
 ('imputer', SimpleImputer(strategy="median")),
 ('attribs_adder', CombinedAttributesAdder()),
 ('std_scaler', StandardScaler())])

full_pipeline = ColumnTransformer([
 ("num", num_pipeline, num_attribs),
 ("cat", OneHotEncoder(), cat_attribs)])

Y_test_prepared = full_pipeline.fit_transform(Y_test)

3-evaluate the final model on the test set

In [49]:
# CODE HERE
final_model = grid_search.best_estimator_
final_predictions = final_model.predict(Y_test_prepared)
final_mse = mean_squared_error(Y_test_labels, final_predictions)
final_rmse = np.sqrt(final_mse)
final_rmse

69661.43722450854

# compute a 95% confidence interval for the generalization error 
*using scipy.stats.t.interval():*

In [50]:
from scipy import stats

In [54]:
# CODE HERE
squared_errors = (final_predictions - Y_test_labels)
stats.t.interval(.95, len(squared_errors) - 1, loc=squared_errors.mean(), scale=stats.sem(squared_errors))

(20853.069289142288, 24869.461524811202)

# Great Job!
# #shAI_Club